<a href="https://colab.research.google.com/github/orel509/AttacksonImplementationsCourseBook/blob/master/Labs/Lecture5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Colab support
!wget https://raw.githubusercontent.com/orel509/AttacksonImplementationsCourseBook/master/Labs/hamming_weight.py
!wget https://raw.githubusercontent.com/orel509/AttacksonImplementationsCourseBook/master/Labs/aes_scripts/aes_lib.py

In [44]:
import matplotlib.pyplot as plt
import numpy as np
from hamming_weight import hamming_weight
#from aes_scripts.aes_crypt_8bit import aes_crypt_8bit
#from aes_scripts.aes_crypt_8bit_and_leak import aes_crypt_8bit_and_leak
from aes_lib import aes_crypt_8bit, aes_crypt_8bit_and_leak

!pip install -U -q bokeh
from bokeh.plotting import figure, show
from bokeh.layouts import row
from bokeh.models import Range1d
from bokeh.io import output_notebook
# Call once to configure Bokeh to display plots inline in the notebook.
output_notebook()

Loading BokehJS ...

In [45]:
#  Make sure the matlab AES scripts are in the path
# #
#  Create two 128-bit plaintexts (exactly 16 byte)
pt1 = []
pt2 = []
for b1 in bytes('Attack at 12:56!', 'utf-8'):
  pt1 += [b1]
for b2 in bytes('Attack at 12:57!', 'utf-8'):
  pt2 += [b2]

plaintext_1 = np.uint8(pt1)
plaintext_1_1 = np.asmatrix(plaintext_1)
plaintext_2 = np.uint8(pt2)
plaintext_2_2 = np.asmatrix(plaintext_2)

In [46]:
#  how many bits are different between the two?
res = hamming_weight(np.bitwise_xor(plaintext_1_1, plaintext_2_2))
print(res)

1.0


In [47]:
#  Create a key
key_bytes = []
for byte in bytes('1234512345123456', 'utf-8'):
  key_bytes += [byte]

key = np.uint8(key_bytes)
key = np.asmatrix(key)

In [48]:
ENCRYPT = 1
DECRYPT = 0
#  Encrypt the two plaintexts
ciphertext_1 = aes_crypt_8bit(plaintext_1_1, key, ENCRYPT)
ciphertext_2 = aes_crypt_8bit(plaintext_2_2, key, ENCRYPT)

In [49]:
#  even though the plaintexts were very similar...
print(plaintext_1)
print(plaintext_2)
#  ... the ciphertexts are very different
print(ciphertext_1)
print(ciphertext_2)

[ 65 116 116  97  99 107  32  97 116  32  49  50  58  53  54  33]
[ 65 116 116  97  99 107  32  97 116  32  49  50  58  53  55  33]
[[ 42 180 180  19 227 238 187 238 237 110   2   6 107 141 160 183]]
[[ 10 223  79  16  74 212 232 141  41  91 134  32 110 141  36 110]]


In [50]:
#  how many bits are different between the two?
print(hamming_weight(np.bitwise_xor(ciphertext_1, ciphertext_2)))

52.0


In [51]:
#  Decrypt the two ciphertexts
decrypted_1 =  aes_crypt_8bit(ciphertext_1, key, DECRYPT)
decrypted_2 =  aes_crypt_8bit(ciphertext_2, key, DECRYPT)

In [52]:
#  Did we get the plaintext again?
print(plaintext_1)
print(plaintext_2)
print(decrypted_1)
print(decrypted_2)

[ 65 116 116  97  99 107  32  97 116  32  49  50  58  53  54  33]
[ 65 116 116  97  99 107  32  97 116  32  49  50  58  53  55  33]
[[ 65 116 116  97  99 107  32  97 116  32  49  50  58  53  54  33]]
[[ 65 116 116  97  99 107  32  97 116  32  49  50  58  53  55  33]]


In [53]:
#  Look at the internals of AES now
[ciphertext_1, state_1, _, leak_1] = aes_crypt_8bit_and_leak(plaintext_1_1, key, ENCRYPT)
[ciphertext_2, state_2, _, leak_2] = aes_crypt_8bit_and_leak(plaintext_2_2, key, ENCRYPT)

In [54]:
shape_1 = np.shape(state_1)
dh_1 = shape_1[0]
dw_1 = shape_1[2]
p1 = figure(title='Plaintext 1 leaks', tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], plot_height=600, plot_width=300)
p1.image(image=[np.squeeze(state_1)], x=0, y=0, dw=dw_1, dh=dh_1, palette="Turbo256", level="image")
#palette values-https://docs.bokeh.org/en/latest/docs/reference/palettes.html
p1.y_range=Range1d(0, dh_1)
p1.x_range=Range1d(0, dw_1)
p2 = figure(title='Plaintext 2 leaks', tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], plot_height=600, plot_width=300)
p2.image(image=[np.squeeze(state_2)], x=0, y=0, dw=dw_1, dh=dh_1, palette="Turbo256", level="image")
p2.y_range=Range1d(0, dh_1)
p2.x_range=Range1d(0, dw_1)
p3 = figure(title='XOR of Plaintext 1 leaks and Plaintext 2 leaks', tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], plot_height=600, plot_width=300)
p3.image(image=[np.squeeze(np.bitwise_xor(state_1, state_2))], x=0, y=0, dw=dw_1, dh=dh_1, palette="Turbo256", level="image")
p3.y_range=Range1d(0, dh_1)
p3.x_range=Range1d(0, dw_1)

show(row(p1,p3,p2))

In [55]:
#  plot the HW of the difference
p = figure(title='Hamming weight for the xor of plaintext1 and plaintext2 leaks', x_axis_label='Leak line', y_axis_label='Hamming weight', tooltips=[("x", "$x"), ("y", "$y")])
p.vbar(x=range(1, (np.shape(state_1)[0]) + 1), top=hamming_weight(np.bitwise_xor(state_1, state_2)), width=0.5, fill_color='blue')
show(p)